# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [57]:
# DATA MANIPULATION
import os
import copy
import pandas as pd
import numpy as np
from datetime import datetime

# DATA VIZ
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = [8, 5]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# MACHINE LEARNING MODELLING
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler

import warnings
warnings.filterwarnings('ignore')
sns.set(color_codes=True)

In [58]:
POWERCO_COLOURS = ["#0072CE", "#B4B4B3", "#79B8F3", "#FDB927", "#F7941D", "#4CB748", "#2E3192"]
DIVERGENT_COLOUR_GRADIENT = ["#e2f1fc", "#b9dcfa", "#8cc7f7", "#5eb1f3", "#39a0f1", "#0691ef"]
sns.set_palette(POWERCO_COLOURS)

---
## 2. Load data

In [59]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [60]:
def map_column_to_integers(df, column_names):
    for column_name in column_names:
      factorized_values, unique_values = pd.factorize(df[column_name][df[column_name] != 'MISSING'])

      mapping = dict(zip(unique_values, range(len(unique_values))))
      mapping['MISSING'] = -1

      df[column_name] = df[column_name].map(mapping)
    return df

In [61]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


In [62]:
df.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14606 non-null  object        
 1   channel_sales                   14606 non-null  object        
 2   cons_12m                        14606 non-null  int64         
 3   cons_gas_12m                    14606 non-null  int64         
 4   cons_last_month                 14606 non-null  int64         
 5   date_activ                      14606 non-null  datetime64[ns]
 6   date_end                        14606 non-null  datetime64[ns]
 7   date_modif_prod                 14606 non-null  datetime64[ns]
 8   date_renewal                    14606 non-null  datetime64[ns]
 9   forecast_cons_12m               14606 non-null  float64       
 10  forecast_cons_year              14606 non-null  int64         
 11  fo

In [64]:
client_date_cols = [
    'date_activ',
    'date_end',
    'date_modif_prod',
    'date_renewal'
]
object_cols = df.select_dtypes(include='object').columns.difference(['id'] + client_date_cols)

for col in object_cols:
  print(f"{col}: {df[col].unique()}\n")

channel_sales: ['foosdfpfkusacimwkcsosbicdxkicaua' 'MISSING'
 'lmkebamcaaclubfxadlmueccxoimlema' 'usilxuppasemubllopkaafesmlibmsdf'
 'ewpakwlliwisiwduibdlfmalxowmwpci' 'epumfxlbckeskwekxbiuasklxalciiuu'
 'sddiedcslfslkckwlfkdpoeeailfpeds' 'fixdbufsefwooaasfcxdxadsiekoceaa']

has_gas: ['t' 'f']

origin_up: ['lxidpiddsbxsbosboudacockeimpuepw' 'kamkkxfxxuwbdslkwifmmcsiusiuosws'
 'ldkssxwpmemidmecebumciepifcamkci' 'MISSING'
 'usapbepcfoloekilkwsdiboslwaxobdp' 'ewxeelcelemmiwuafmddpobolfuxioce']



In [65]:
df = map_column_to_integers(df, object_cols)

In [66]:
for col in object_cols:
  print(f"{col}: {df[col].unique()}\n")

channel_sales: [ 0 -1  1  2  3  4  5  6]

has_gas: [0 1]

origin_up: [ 0  1  2 -1  3  4]



In [67]:
cp_df = copy.deepcopy(df)
cp_df[client_date_cols] = cp_df[client_date_cols].apply(lambda x: x.astype(int) / 10**9)
cp_df.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [68]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [69]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [70]:
diff = pd.merge(diff, df[['id', 'churn']], on='id', how='inner')
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,churn
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916,0
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779,0
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000,0
3,00114d74e963e47177db89bc70108537,-0.003994,-0.000001,0
4,0013f326a839a2f6ad87a1859952d227,-0.006171,0.000000,0


In [71]:
fig = px.scatter(
    diff,
    x = ['offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power'],
    y = 'churn',
    title='<b>Difference between off-peak prices in December and preceding January<b>',
)

fig.update_layout(
    xaxis_title='Consumption Difference',
    yaxis_title='Churn',
    legend=dict(x=0, y=1, bgcolor='rgba(0, 0, 0, 0)', title='Consumption'),
    annotations=[
    dict(
        x=0,
        y=1.1,
        xref='paper',
        yref='paper',
        text='No Apparent Relationship with Churn',
        showarrow=False,
        font=dict(size=16)
      )
    ],
)

fig.for_each_trace(
    lambda t: t.update(name='Energy') if t.name == 'offpeak_diff_dec_january_energy'
    else t.update(name='Power')
)


fig.show()

In [72]:
df = pd.merge(df, diff[['id', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']], on='id', how='inner')
cp_df = pd.merge(cp_df, diff[['id', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']], on='id', how='inner')
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,0,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,-1,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,0,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0,-0.004670,0.177779
3,bba03439a292a1e166f80264c16191cb,1,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0,-0.004547,0.177779
4,149d57cf92fc41cf94415803a877cb4b,-1,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0,-0.006192,0.162916


In the previous notebook, I tried to think of possible features that can be used to predict churn to no avail. The suggestion above from Estelle sounded clever, but really doesn't seem like it will have any predictive power.

I am thus going to throw all of these variables in a Random Forest classifier and to see if it will be able to identify any important features. This is another attempt at feature selection as the random forest is capable of identifying nonlinear relationships.

# 4. Baseline Model

## Model Helpers

In [73]:
def basic_rf_eval(model, test_data, true_y):
  pred_y = model.predict(test_data)

  accuracy = accuracy_score(true_y, pred_y)
  confusion_matrix_ = confusion_matrix(true_y, pred_y)
  classification_report_ = classification_report(true_y, pred_y)

  print(f'Accuracy = {accuracy.round(2)};\n\
    \nConfusion Matrix: \n{confusion_matrix_};\n\
    \nClassification Report:\n{classification_report_}'
  )

In [74]:
def create_feature_importance_df(rf_model):
  feature_list = list(X.columns)
  importances = list(rf_model.feature_importances_)

  feature_importances = [
      (feature, round(importances, 2)) for feature, importances in zip(feature_list, importances)
  ]

  importance_df = pd.DataFrame(feature_importances, columns=['Variable', 'Importance'])

  return importance_df

In [75]:
def view_feature_importances(importance_df):
  fig = px.bar(
      importance_df,
      x='Variable',
      y='Importance',
      labels={'Variable': 'Variable', 'Importance': 'Importance'},
      title='Variable Importances',
      orientation='h'
  )

  fig.update_layout(
      xaxis={'categoryorder': 'total descending'},
      yaxis_title='Importances',
      xaxis_title='Variables',
      title={'x': 0.5}
  )

  fig.show()

## Iterating on Models

In [76]:
X = cp_df.drop(columns=['id', 'churn'])
y = cp_df['churn']

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
baseline_rf_model = RandomForestClassifier(random_state=42)
baseline_rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [78]:
basic_rf_eval(baseline_rf_model, X_test, y_test)

Accuracy = 0.9;
    
Confusion Matrix: 
[[2614    3]
 [ 287   18]];
    
Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2617
           1       0.86      0.06      0.11       305

    accuracy                           0.90      2922
   macro avg       0.88      0.53      0.53      2922
weighted avg       0.90      0.90      0.86      2922



In [79]:
baseline_importance_df = create_feature_importance_df(baseline_rf_model)
view_feature_importances(baseline_importance_df)

This is a very bad model as it has extremely low sensitivity to the actual class we're interested in i.e. class 1(Churn). I want to assume that the imbalanced nature of the data is one of the major reasons why. Intuitively, I think a sampling strategy can work. There's also weighting classes and using the **BalancedRandomForestClassifier**(both of which I trust less) that I'm not sure of how it works. Let's play around and see what we get.

If I can achieve reasonable improve by just samples, then I'd be very happy. To improve the model further, I think feature engineering and hyperparameter tuning will help. In fact, I believe the order should be: feature engineering(which I tried in the previous notebook), deal with the balance of the dataset then finally tune the hyperparameters.

In [80]:
balanced_baseline = BalancedRandomForestClassifier(random_state=42)
balanced_baseline.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=42)

In [81]:
basic_rf_eval(balanced_baseline, X_test, y_test)

Accuracy = 0.67;
    
Confusion Matrix: 
[[1777  840]
 [ 133  172]];
    
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.68      0.79      2617
           1       0.17      0.56      0.26       305

    accuracy                           0.67      2922
   macro avg       0.55      0.62      0.52      2922
weighted avg       0.85      0.67      0.73      2922



Now we're fighting the battle of tilting the scales... The same way the recall for Churn has improved is the same way that we've increase Type I and II errors😆😆😆😆😆

In [82]:
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)
undersampled_baseline = RandomForestClassifier(random_state=42)
undersampled_baseline.fit(X_resampled, y_resampled)
basic_rf_eval(undersampled_baseline, X_resampled, y_resampled)

Accuracy = 1.0;
    
Confusion Matrix: 
[[1419    0]
 [   0 1419]];
    
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1419
           1       1.00      1.00      1.00      1419

    accuracy                           1.00      2838
   macro avg       1.00      1.00      1.00      2838
weighted avg       1.00      1.00      1.00      2838



100% everything??? Kanjani? Just how??? 😆😆😆😆😆

Perhaps the sampling strategy was wrong, or the model is just overfit. No ways this is good model.

In [83]:
undersampled_importance_df = create_feature_importance_df(undersampled_baseline)
view_feature_importances(undersampled_importance_df)

**GOODNESS GRACIOUS, NO FEATURE IS USEFUL** This is no surprise though, we couldn't get anything useful out of a fawl model

# 3. Back To Feature Engineering

## Average price changes across periods

We can now enhance the feature that our colleague made by calculating the average price changes across individual periods, instead of the entire year.

This feature may be useful because it adds more granularity to the existing feature that my colleague found to be useful. Instead of looking at differences across an entire year, we have now created features that look at mean average price differences across different time periods (off_peak, peak, mid_peak). The dec-jan feature may reveal macro patterns that occur over an entire year, whereas inter-time-period features may reveal patterns on a micro scale between months.

In [84]:
price_cols = [
    'price_off_peak_var',
    'price_peak_var',
    'price_mid_peak_var',
    'price_off_peak_fix',
    'price_peak_fix',
    'price_mid_peak_fix'
]

column_pairs = {
    'off_peak_peak_var_mean_diff': ['price_off_peak_var', 'price_peak_var'],
    'peak_mid_peak_var_mean_diff': ['price_peak_var', 'price_mid_peak_var'],
    'off_peak_mid_peak_var_mean_diff': ['price_off_peak_var', 'price_mid_peak_var'],
    'off_peak_peak_fix_mean_diff': ['price_off_peak_fix', 'price_peak_fix'],
    'peak_mid_peak_fix_mean_diff': ['price_peak_fix', 'price_mid_peak_fix'],
    'off_peak_mid_peak_fix_mean_diff': ['price_off_peak_fix', 'price_mid_peak_fix']
}

avg_price_cols = [
    'id',
    'off_peak_peak_var_mean_diff',
    'peak_mid_peak_var_mean_diff',
    'off_peak_mid_peak_var_mean_diff',
    'off_peak_peak_fix_mean_diff',
    'peak_mid_peak_fix_mean_diff',
    'off_peak_mid_peak_fix_mean_diff'
]

In [85]:
mean_prices = price_df.groupby('id')[price_cols].mean().reset_index()

for new_col, col_pair in column_pairs.items():
    mean_prices[new_col] = mean_prices[col_pair[0]] - mean_prices[col_pair[1]]

df = pd.merge(df, mean_prices[avg_price_cols], on='id')
df.head()


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,off_peak_peak_var_mean_diff,peak_mid_peak_var_mean_diff,off_peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff
0,24011ae4ebbe3035111d65fa7c15bc57,0,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,4.423670e+01,1,0.020057,3.700961,0.024038,0.034219,0.058257,18.590255,7.450670,26.040925
1,d29c2c54acc38ff3c0614d0a653813dd,-1,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000000e+00,0,-0.003767,0.177779,0.142485,0.007124,0.149609,44.311375,0.000000,44.311375
2,764c75f661154dac3a6c254cd082ea7d,0,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000e+00,0,-0.004670,0.177779,0.082090,0.088421,0.170512,44.385450,0.000000,44.385450
3,bba03439a292a1e166f80264c16191cb,1,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0,-0.004547,0.177779,0.151210,0.000000,0.151210,44.400265,0.000000,44.400265
4,149d57cf92fc41cf94415803a877cb4b,-1,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,4.860000e-10,0,-0.006192,0.162916,0.020536,0.030773,0.051309,16.275263,8.137629,24.412893


In [86]:
mean_prices.head()

,id,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,off_peak_peak_var_mean_diff,peak_mid_peak_var_mean_diff,off_peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff
0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,0.020545,0.030633,0.051178,16.280694,8.140345,24.421038
1,0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.000000,44.385450,0.000000,0.000000,0.146426,0.000000,0.146426,44.385450,0.000000,44.385450
2,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.000000,45.319710,0.000000,0.000000,0.181558,0.000000,0.181558,45.319710,0.000000,45.319710
3,0010ee3855fdea87602a5b7aba8e42de,0.118757,0.098292,0.069032,40.647427,24.388455,16.258971,0.020465,0.029260,0.049725,16.258972,8.129484,24.388456
4,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0.147926,0.000000,0.147926,44.266930,0.000000,44.266930


## Max price changes across periods and months

Another way we can enhance the feature from our colleague is to look at the maximum change in prices across periods and months.

I thought that calculating the maximum price change between months and time periods would be a good feature to create because I was trying to think from the perspective of a PowerCo client. As a Utilities customer, there is nothing more annoying than sudden price changes between months, and a large increase in prices within a short time span would be an influencing factor in causing me to look at other utilities providers for a better deal. Since we are trying to predict churn for this use case, I thought this would be an interesting feature to include.

In [87]:
# Bad practise this one, but naming variables is hard. LOL!
column_pairs = {
    'off_peak_peak_var_mean_diff': ['price_off_peak_var', 'price_peak_var'],
    'peak_mid_peak_var_mean_diff': ['price_peak_var', 'price_mid_peak_var'],
    'off_peak_mid_peak_var_mean_diff': ['price_off_peak_var', 'price_mid_peak_var'],
    'off_peak_peak_fix_mean_diff': ['price_off_peak_fix', 'price_peak_fix'],
    'peak_mid_peak_fix_mean_diff': ['price_peak_fix', 'price_mid_peak_fix'],
    'off_peak_mid_peak_fix_mean_diff': ['price_off_peak_fix', 'price_mid_peak_fix']
}

mean_prices_by_month = price_df.groupby(['id', 'price_date'])[price_cols].mean().reset_index()

for new_col, col_pair in column_pairs.items():
    mean_prices_by_month[new_col] = mean_prices_by_month[col_pair[0]] - mean_prices_by_month[col_pair[1]]

Now calculating maximum monthly differences across time periods

In [88]:
max_diff_across_periods_months = mean_prices_by_month.groupby(['id']).agg({
    'off_peak_peak_var_mean_diff': 'max',
    'peak_mid_peak_var_mean_diff': 'max',
    'off_peak_mid_peak_var_mean_diff': 'max',
    'off_peak_peak_fix_mean_diff': 'max',
    'peak_mid_peak_fix_mean_diff': 'max',
    'off_peak_mid_peak_fix_mean_diff': 'max'
}).reset_index().rename(
    columns={
        'off_peak_peak_var_mean_diff': 'off_peak_peak_var_max_monthly_diff',
        'peak_mid_peak_var_mean_diff': 'peak_mid_peak_var_max_monthly_diff',
        'off_peak_mid_peak_var_mean_diff': 'off_peak_mid_peak_var_max_monthly_diff',
        'off_peak_peak_fix_mean_diff': 'off_peak_peak_fix_max_monthly_diff',
        'peak_mid_peak_fix_mean_diff': 'peak_mid_peak_fix_max_monthly_diff',
        'off_peak_mid_peak_fix_mean_diff': 'off_peak_mid_peak_fix_max_monthly_diff'
    }
)

In [89]:
columns = [
    'id',
    'off_peak_peak_var_max_monthly_diff',
    'peak_mid_peak_var_max_monthly_diff',
    'off_peak_mid_peak_var_max_monthly_diff',
    'off_peak_peak_fix_max_monthly_diff',
    'peak_mid_peak_fix_max_monthly_diff',
    'off_peak_mid_peak_fix_max_monthly_diff'
]

df = pd.merge(df, max_diff_across_periods_months[columns], on='id')
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,off_peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff,off_peak_peak_var_max_monthly_diff,peak_mid_peak_var_max_monthly_diff,off_peak_mid_peak_var_max_monthly_diff,off_peak_peak_fix_max_monthly_diff,peak_mid_peak_fix_max_monthly_diff,off_peak_mid_peak_fix_max_monthly_diff
0,24011ae4ebbe3035111d65fa7c15bc57,0,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.058257,18.590255,7.450670,26.040925,0.060550,0.085483,0.146033,44.266930,8.145775,44.26693
1,d29c2c54acc38ff3c0614d0a653813dd,-1,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.149609,44.311375,0.000000,44.311375,0.151367,0.085483,0.151367,44.444710,0.000000,44.44471
2,764c75f661154dac3a6c254cd082ea7d,0,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.170512,44.385450,0.000000,44.385450,0.084587,0.089162,0.172468,44.444710,0.000000,44.44471
3,bba03439a292a1e166f80264c16191cb,1,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.151210,44.400265,0.000000,44.400265,0.153133,0.000000,0.153133,44.444710,0.000000,44.44471
4,149d57cf92fc41cf94415803a877cb4b,-1,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.051309,16.275263,8.137629,24.412893,0.022225,0.033743,0.055866,16.291555,8.145775,24.43733


## Tenure

In [90]:
df['tenure'] = ((df['date_end'] - df['date_activ'])/ np.timedelta64(1, 'Y')).astype(int)
churn_by_tenure = df.groupby(['tenure']).agg({'churn': 'mean'}).sort_values(by='churn', ascending=False)
churn_by_tenure.head()

,churn
tenure,
3,0.143713
2,0.133080
4,0.125756
13,0.095238
5,0.085425


In [91]:
fig = px.bar(
      churn_by_tenure,
      x=churn_by_tenure.index,
      y='churn',
      labels={'churn': 'Average Churn Rate', 'tenure': 'Customer Tenure'},
      title="<b>Average Churn Rate per Tenure<b>",
    )

fig.add_annotation(
    go.layout.Annotation(
        x=0,
        y=1.1,
        text='Churn Seems More Prevalent on the Extremes',
        showarrow=False,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="top",
        font=dict(size=16),
        align='left'
    )
)

fig.show()

We can see that companies who have only been a client for 4 or less months are much more likely to churn compared to companies that have been a client for longer. Interestingly, the difference between 4 and 5 months is about 4%, which represents a large jump in likelihood for a customer to churn compared to the other differences between ordered tenure values. Perhaps this reveals that getting a customer to over 4 months tenure is actually a large milestone with respect to keeping them as a long term customer.

This is an interesting feature to keep for modelling because clearly how long you've been a client, has a influence on the chance of a client churning

### Transforming dates into months

- months_activ = Number of months active until reference date (Jan 2016)
- months_to_end = Number of months of the contract left until reference date (Jan 2016)
- months_modif_prod = Number of months since last modification until reference date (Jan 2016)
- months_renewal = Number of months since last renewal until reference date (Jan 2016)

In [92]:
def convert_months(reference_date, df, column):
    time_delta = reference_date - df[column]
    months = (time_delta / np.timedelta64(1, 'M')).astype(int)
    return months

In [93]:
reference_date = datetime(2016, 1, 1)

df['months_activ'] = convert_months(reference_date, df, 'date_activ')
df['months_to_end'] = -convert_months(reference_date, df, 'date_end')
df['months_modif_prod'] = convert_months(reference_date, df, 'date_modif_prod')
df['months_renewal'] = convert_months(reference_date, df, 'date_renewal')

In [94]:
reference_date = pd.to_datetime('now')

date_columns = {
    'months_active': 'date_activ',
    'months_to_end': 'date_end',
    'months_modif_prod': 'date_modif_prod',
    'months_renewal': 'date_renewal'
}

for new_col, column_name in date_columns.items():
    df[new_col] = convert_months(reference_date, df, column_name)

Dates as a datetime object are not useful for a predictive model, so we needed to use the datetimes to create some other features that may hold some predictive power.

Using intuition, you could assume that a client who has been an active client of PowerCo for a longer amount of time may have more loyalty to the brand and is more likely to stay. Whereas a newer client may be more volatile. Hence the addition of the months_activ feature.

As well as this, if we think from the perspective of a client with PowerCo, if you're coming toward the end of your contract with PowerCo your thoughts could go a few ways. You could be looking for better deals for when your contract ends, or you might want to see out your contract and sign another one. One the other hand if you've only just joined, you may have a period where you're allowed to leave if you're not satisfied. Furthermore, if you're in the middle of your contract, their may be charges if you wanted to leave, deterring clients from churning mid-way through their agreement. So, I think months_to_end will be an interesting feature because it may reveal patterns and behaviours about timing of churn.

My belief is that if a client has made recent updates to their contract, they are more likely to be satisfied or at least they have received a level of customer service to update or change their existing services. I believe this to be a positive sign, they are an engaged customer, and so I believe months_modif_prod will be an interesting feature to include because it shows the degree of how 'engaged' a client is with PowerCo.

Finally the number of months since a client last renewed a contract I believe will be an interesting feature because once again, it shows the degree to which that client is engaged. It also goes a step further than just engagement, it shows a level of commitment if a client renews their contract. For this reason, I believe months_renewal will be a good feature to include.

In [95]:
cols_to_drop = [
    'date_activ',
    'date_end',
    'date_modif_prod',
    'date_renewal'
] # services no longer needed

df = df.drop(columns=cols_to_drop)
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,...,off_peak_mid_peak_var_max_monthly_diff,off_peak_peak_fix_max_monthly_diff,peak_mid_peak_fix_max_monthly_diff,off_peak_mid_peak_fix_max_monthly_diff,tenure,months_activ,months_to_end,months_modif_prod,months_renewal,months_active
0,24011ae4ebbe3035111d65fa7c15bc57,0,0,54946,0,0.00,0,0.0,1.78,0.114481,...,0.146033,44.266930,8.145775,44.26693,3,30,91,99,103,127
1,d29c2c54acc38ff3c0614d0a653813dd,-1,4660,0,0,189.95,0,0.0,16.27,0.145711,...,0.151367,44.444710,0.000000,44.44471,7,76,89,173,101,173
2,764c75f661154dac3a6c254cd082ea7d,0,544,0,0,47.96,0,0.0,38.72,0.165794,...,0.172468,44.444710,0.000000,44.44471,6,68,93,165,105,165
3,bba03439a292a1e166f80264c16191cb,1,1584,0,0,240.04,0,0.0,19.83,0.146694,...,0.153133,44.444710,0.000000,44.44471,6,69,94,166,106,166
4,149d57cf92fc41cf94415803a877cb4b,-1,4425,0,526,445.75,526,0.0,131.73,0.116900,...,0.055866,16.291555,8.145775,24.43733,6,71,95,168,107,168


### Transforming categorical data

A predictive model cannot accept categorical or `string` values, hence as a data scientist you need to encode categorical features into numerical representations in the most compact and discriminative way possible.

The simplest method is to map each category to an integer (label encoding), however this is not always appropriate beecause it then introduces the concept of an order into a feature which may not inherently be present `0 < 1 < 2 < 3 ...`

Another way to encode categorical features is to use `dummy variables` AKA `one hot encoding`. This create a new feature for every unique value of a categorical column, and fills this column with either a 1 or a 0 to indicate that this company does or does not belong to this category.

#### channel_sales

In [96]:
df['channel_sales'] = df['channel_sales'].astype('category')
df['channel_sales'].value_counts()

0     6754
-1    3725
1     1843
2     1375
3      893
5       11
4        3
6        2
Name: channel_sales, dtype: int64

In [97]:
channel_mapping = {
    0: 'ch0',
    1: 'ch1',
    2: 'ch2',
    3: 'ch3',
    4: 'ch4',
    5: 'ch5',
    6: 'ch6',
    -1: 'missing_channel'
}

df['channel_sales'] = df['channel_sales'].replace(channel_mapping)
df = pd.get_dummies(df, columns=['channel_sales'], prefix='')
df.head()


,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_renewal,months_active,_missing_channel,_ch0,_ch1,_ch2,_ch3,_ch4,_ch5,_ch6
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,103,127,0,1,0,0,0,0,0,0
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,101,173,1,0,0,0,0,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,105,165,0,1,0,0,0,0,0,0
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,106,166,0,0,1,0,0,0,0,0
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,107,168,1,0,0,0,0,0,0,0


In [99]:
df.rename(columns={
    '_missing_channel': 'missing_channel',
    '_ch0': 'ch0',
    '_ch1': 'ch1',
    '_ch2': 'ch2',
    '_ch3': 'ch3',
    '_ch4': 'ch4',
    '_ch5': 'ch5',
    '_ch6': 'ch6'}, inplace=True)

df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_renewal,months_active,missing_channel,ch0,ch1,ch2,ch3,ch4,ch5,ch6
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,103,127,0,1,0,0,0,0,0,0
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,101,173,1,0,0,0,0,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,105,165,0,1,0,0,0,0,0,0
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,106,166,0,0,1,0,0,0,0,0
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,107,168,1,0,0,0,0,0,0,0


### Transforming numerical data

In the previous exercise we saw that some variables were highly skewed. The reason why we need to treat skewness is because some predictive models have inherent assumptions about the distribution of the features that are being supplied to it. Such models are called `parametric` models, and they typically assume that all variables are both independent and normally distributed.

Skewness isn't always a bad thing, but as a rule of thumb it is always good practice to treat highly skewed variables because of the reason stated above, but also as it can improve the speed at which predictive models are able to converge to its best solution.

There are many ways that you can treat skewed variables. You can apply transformations such as:
- Square root
- Cubic root
- Logarithm

to a continuous numeric column and you will notice the distribution changes. For this use case we will use the 'Logarithm' transformation for the positively skewed features.

<b>Note:</b> We cannot apply log to a value of 0, so we will add a constant of 1 to all the values

First I want to see the statistics of the skewed features, so that we can compare before and after transformation

In [104]:
skewed = [
    'cons_12m',
    'cons_gas_12m',
    'cons_last_month',
    'forecast_cons_12m',
    'forecast_cons_year',
    'forecast_discount_energy',
    'forecast_meter_rent_12m',
    'forecast_price_energy_off_peak',
    'forecast_price_energy_peak',
    'forecast_price_pow_off_peak'
]

df[skewed].describe().T

,count,mean,std,min,25%,50%,75%,max
cons_12m,14606.0,4.223939,0.884515,0.0,3.754023,4.149727,4.610285,6.792889
cons_gas_12m,14606.0,0.779244,1.717071,0.0,0.000000,0.000000,0.000000,6.618528
cons_last_month,14606.0,2.264646,1.769305,0.0,0.000000,2.899547,3.529430,5.887169
forecast_cons_12m,14606.0,2.962177,0.683592,0.0,2.695477,3.046836,3.380716,4.918575
forecast_cons_year,14606.0,1.784610,1.584986,0.0,0.000000,2.498311,3.242231,5.243970
forecast_discount_energy,14606.0,0.966726,5.108289,0.0,0.000000,0.000000,0.000000,30.000000
forecast_meter_rent_12m,14606.0,1.517203,0.571481,0.0,1.235023,1.296555,2.120673,2.778376
forecast_price_energy_off_peak,14606.0,0.137283,0.024623,0.0,0.116340,0.143166,0.146348,0.273963
forecast_price_energy_peak,14606.0,0.050491,0.049037,0.0,0.000000,0.084138,0.098837,0.195975
forecast_price_pow_off_peak,14606.0,43.130056,4.485988,0.0,40.606701,44.311378,44.311378,59.266378


In [100]:
cols_to_transform = [
    "cons_12m",
    "cons_gas_12m",
    "cons_last_month",
    "forecast_cons_12m",
    "forecast_cons_year",
    "forecast_meter_rent_12m",
    "imp_cons"
]

for col_name in cols_to_transform:
    df[col_name] = np.log10(df[col_name] + 1)

df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_renewal,months_active,missing_channel,ch0,ch1,ch2,ch3,ch4,ch5,ch6
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,...,103,127,0,1,0,0,0,0,0,0
1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2.280920,0.000000,0.0,1.237292,0.145711,0.000000,...,101,173,1,0,0,0,0,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,1.689841,0.000000,0.0,1.599009,0.165794,0.087899,...,105,165,0,1,0,0,0,0,0,0
3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2.382089,0.000000,0.0,1.318689,0.146694,0.000000,...,106,166,0,0,1,0,0,0,0,0
4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2.650065,2.721811,0.0,2.122969,0.116900,0.100015,...,107,168,1,0,0,0,0,0,0,0


In [103]:
df[cols_to_transform].describe().T

,count,mean,std,min,25%,50%,75%,max
cons_12m,14606.0,4.223939,0.884515,0.0,3.754023,4.149727,4.610285,6.792889
cons_gas_12m,14606.0,0.779244,1.717071,0.0,0.000000,0.000000,0.000000,6.618528
cons_last_month,14606.0,2.264646,1.769305,0.0,0.000000,2.899547,3.529430,5.887169
forecast_cons_12m,14606.0,2.962177,0.683592,0.0,2.695477,3.046836,3.380716,4.918575
forecast_cons_year,14606.0,1.784610,1.584986,0.0,0.000000,2.498311,3.242231,5.243970
forecast_meter_rent_12m,14606.0,1.517203,0.571481,0.0,1.235023,1.296555,2.120673,2.778376
imp_cons,14606.0,1.240750,1.138322,0.0,0.000000,1.584275,2.289990,4.177357


In [102]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df["cons_12m"], nbinsx=50, name="cons_12m"))
fig.add_trace(go.Histogram(x=df[df["has_gas"] == 1]["cons_gas_12m"], nbinsx=50, name="cons_gas_12m for has_gas=1"))
fig.add_trace(go.Histogram(x=df["cons_last_month"], nbinsx=50, name="cons_last_month"))

fig.update_layout(title="Histograms of Consumption",
                  xaxis_title="Consumption",
                  yaxis_title="Count",
                  barmode="overlay")

fig.update_traces(opacity=0.75)

fig.show()